# Nightclubs and bars

In [1]:
from dotenv import load_dotenv
import json, requests
from bson import ObjectId
from folium import Map, Marker, Icon, FeatureGroup, LayerControl, Choropleth
from folium.plugins import HeatMap
from folium.vector_layers import Circle, Polygon
import pandas as pd

In [2]:
from pymongo import MongoClient
client = MongoClient()
db = client.get_database('companies_project')

In this filter I will check the number of bars and nightclubs in a 10-kilometer radius around each of the offices, through the Foursquare Places API, in order to ensure that the candidates are surrounded by a proper leisure offer for our employees. 

In [3]:
offices = list(db.companies_usa_filt1.find({}, {'_id':1, 'offices.latitude':1, 'offices.longitude':1,}))

In [5]:
load_dotenv()
foursq_api_id = os.getenv('foursquare_id')
foursq_api_k = os.getenv('foursquare_key')

In [7]:
for office in offices[1:]:
    # Get latitude, longitude and id of the document
    latitude = office['offices']['latitude']
    longitude = office['offices']['longitude']
    id = ObjectId(office['_id'])

    # Set the requesto to Foursquare API, with its parameters (keys, latitude&longitude, 10,000 radius, 'Nightlife Spot' as categoryId and version_date)
    url = 'https://api.foursquare.com/v2/venues/search'
    params = dict(
        client_id = f"{foursq_api_id}",
        client_secret = f"{foursq_api_k}",
        ll = f"{latitude},{longitude}",
        radius = '10000',
        categoryId = '4d4b7105d754a06376d81259',
        v = 20201115)
    
    # Do the request and get the data
    resp = requests.get(url=url, params=params)
    data = json.loads(resp.text)
    # Organize the data as dict of dicts, where the keys will go from 0 to the number of results - 1 and the values will be each of the found venues
    data_format = {}
    index = 0
    for venue in data['response']['venues']:
        data_format[str(index)] = venue
        index += 1
    data_format['n_venues'] = len(data_format)

    # Update the document with a new section, called 'nightlife_spots'
    db.companies_usa_filt1.update_one({'_id':id}, {'$set':{'offices.nightlife_spots': data_format}})

Let's check how many of those candidates are surrounded by 25 or more venues.

In [16]:
offices_2 = list(db.companies_usa_filt1.find({'offices.nightlife_spots.n_venues':{'$gte':25}}))

In [17]:
len(offices_2)

1382

There are 1382 candidates that have 25 or more nightclubs and bars in a 10-kilometer radius. Let's save them into the database as a new collection and move onto the next filter.

In [18]:
db.companies_usa_filt2_c.insert_many(offices_2)

# Map

In [4]:
offices_filter1 = list(db.companies_usa_filt1.find({}, {'offices':1}))
to_df = {'_id': [], 'latitude': [], 'longitude': [], 'n_venues': []}

for office in offices_filter1:
    to_df['_id'].append(office['_id'])
    to_df['latitude'].append(office['offices']['latitude'])
    to_df['longitude'].append(office['offices']['longitude'])
    to_df['n_venues'].append(office['offices']['nightlife_spots']['n_venues'])

In [5]:
night_ven_df = pd.DataFrame.from_dict(to_df)
night_ven_df.head()

,_id,latitude,longitude,n_venues
0,5faff5b2d77f2daa19a53e7f,40.723731,-73.996431,30
1,5faff5b2d77f2daa19a53e81,37.764726,-122.394523,30
2,5faff5b2d77f2daa19a53e89,37.789634,-122.404052,30
3,5faff5b2d77f2daa19a53e8c,40.757929,-73.985506,30
4,5faff5b2d77f2daa19a53e95,40.757725,-73.986011,30


In [6]:
m_heat = Map(location=[37.0902,-95.7129],zoom_start=4)

all = HeatMap(data=night_ven_df[["latitude","longitude", 'n_venues']], name='Offices', max_val=30, radius=10)
fg = FeatureGroup(name="Offices")
all.add_to(fg)
fg.add_to(m_heat)

selected = HeatMap(data=night_ven_df[night_ven_df['n_venues'] >= 25][["latitude","longitude", 'n_venues']], name='Selected', max_val=30, radius=10)
fg_sel = FeatureGroup(name="Selected")
selected.add_to(fg_sel)
fg_sel.add_to(m_heat)

LayerControl(collapsed=False).add_to(m_heat)
m_heat